In [25]:
import numpy as np
import pandas as pd

demands = pd.read_csv("demand_list.csv")

In [26]:
demands

,행정동,이름,주소,Latitude,Longitude
0,영등포동,중마루공원,서울 영등포구 영등포로53길 14,37.519790,126.911368
1,영등포동,타임스퀘어,서울 영등포구 영중로 15,37.517075,126.903341
2,영등포동,서울 사회복지대학원 대학교,서울 영등포구 영신로34길 18,37.519528,126.903109
3,영등포동,영등포 중앙어린이공원,서울 영등포구 영등포로35길 13,37.521440,126.903767
4,영등포동,영등포동 주민센터,서울 영등포구 영등포로53길 22,37.520343,126.910682
...,...,...,...,...,...
844,대림3동,견인지역357,서울특별시 영등포구 대림동 779-8,37.500172,126.897178
845,대림3동,견인지역358,서울특별시 영등포구 대림동 777-2,37.495687,126.894759
846,대림3동,견인지역359,서울특별시 영등포구 대림동 777-1,37.497061,126.894673
847,대림3동,견인지역360,서울특별시 영등포구 대림동 736-2,37.495854,126.898507


In [27]:
demands = demands.drop(labels = ["주소", "이름"], axis = 1)

In [28]:
demands["Demand"] = 1

In [29]:
demands_ydp = demands[demands['행정동']=='영등포동']
demands_yu = demands[demands['행정동']=='여의동']
demands_yp2 = demands[demands['행정동']=='양평2동']
demands_ml = demands[demands['행정동']=='문래동']
demands_dl3 = demands[demands['행정동']=='대림3동']

In [30]:
demands_ydp = demands_ydp.drop(labels = ['행정동'], axis = 1)
demands_ydp

,Latitude,Longitude,Demand
0,37.519790,126.911368,1
1,37.517075,126.903341,1
2,37.519528,126.903109,1
3,37.521440,126.903767,1
4,37.520343,126.910682,1
...,...,...,...
761,37.518679,126.909569,1
762,37.518463,126.915407,1
763,37.519434,126.908866,1
764,37.519013,126.914342,1


In [31]:
demands_yu = demands_yu.drop(labels = ['행정동'], axis = 1)
demands_yu

,Latitude,Longitude,Demand
32,37.517694,126.934614,1
33,37.524169,126.926428,1
34,37.526809,126.922284,1
35,37.528310,126.920759,1
36,37.518822,126.927839,1
...,...,...,...
800,37.531218,126.920835,1
801,37.530093,126.922421,1
802,37.529546,126.922923,1
803,37.529714,126.925664,1


In [32]:
demands_yp2 = demands_yp2.drop(labels = ['행정동'], axis = 1)
demands_yp2

,Latitude,Longitude,Demand
110,37.540099,126.892080,1
111,37.531485,126.891855,1
112,37.531829,126.893495,1
113,37.532020,126.895554,1
114,37.531308,126.894989,1
...,...,...,...
814,37.539530,126.896184,1
815,37.536300,126.897864,1
816,37.534551,126.892638,1
817,37.530216,126.894670,1


In [33]:
demands_ml = demands_ml.drop(labels = ['행정동'], axis = 1)
demands_ml

,Latitude,Longitude,Demand
136,37.517110,126.899523,1
137,37.520810,126.889034,1
138,37.516466,126.894020,1
139,37.518240,126.895880,1
140,37.514259,126.897682,1
...,...,...,...
837,37.516109,126.900718,1
838,37.511262,126.891910,1
839,37.511254,126.892321,1
840,37.512915,126.896302,1


In [34]:
demands_dl3 = demands_dl3.drop(labels = ['행정동'], axis = 1)
demands_dl3

,Latitude,Longitude,Demand
165,37.496976,126.902536,1
166,37.498091,126.899611,1
167,37.499689,126.894704,1
168,37.504301,126.896095,1
169,37.504123,126.894823,1
...,...,...,...
844,37.500172,126.897178,1
845,37.495687,126.894759,1
846,37.497061,126.894673,1
847,37.495854,126.898507,1


영등포동에 대한 MCLP 입지선정, 시각화

In [35]:
demand_points = [(x, y, z) for x, y, z in zip(demands_ydp['Longitude'], demands_ydp['Latitude'], demands_ydp['Demand'])]
facility_points = [(x, y) for x, y in zip(demands_ydp['Longitude'], demands_ydp['Latitude'])]

In [36]:
import pulp
from haversine import haversine
import numpy as np

# 목적함수를 최대화하는 것을 목표로 함
problem = pulp.LpProblem("MCLP", pulp.LpMaximize)

# 수요 지점과 시설 입지 후보 지점

# 각 시설의 최대 수요 커버 반경 (km 단위)
coverage_distance = 0.5

# 최종적으로 선정할 입지의 갯수
num_facilities = 2

# Haversine을 이용하여 위경도 좌표를 기준으로 거리를 계산
distances = np.array(
    [
        [haversine((fy, fx), (dy, dx)) for dx, dy, _ in demand_points]
        for fx, fy in facility_points
    ]
)

# 각 시설 입지 지점에 대응하는 이진 변수 (x[i] == 1 : 해당 지점이 선택됨 / x[i] == 0 : 해당 지점이 
# 선택되지 않음
x = pulp.LpVariable.dicts(
    "facility", [(i) for i in range(len(facility_points))], cat="Binary"
)

# 각 입지 지점과 수요 지점의 쌍에 대응하는 이진 변수 딕셔너리를 생성
# 이 딕셔너리는 밑에서 추가할 제약을 바탕으로 해당 입지 지점이 선정되었다면, 계산에서 사용됨
y = pulp.LpVariable.dicts(
    "demand",
    [(i, j) for i in range(len(facility_points)) for j in range(len(demand_points))],
    cat="Binary",
)

# 목적 함수 :  sum y_ij * 가중치를 최대화 함
problem += pulp.lpSum(
    [y[(i, j)] * demand_points[j][2] for i in range(len(facility_points)) for j in range(len(demand_points))]
)

# 제약 1 : 최종적으로 선정할 입지의 갯수는 num_facilities와 동일해야 함
problem += pulp.lpSum([x[i] for i in range(len(facility_points))]) == num_facilities

# 제약 2 : y[(i, j)]가 계산되려면, 앞서 x[i] 후보지가 선정되어야 함
# 제약 3 : i번째 후보지와 j번째 수요지 사이의 거리가 coverage_distance보다 작거나 같아야 수요를 커버 가능
for i in range(len(facility_points)):
    for j in range(len(demand_points)):
        problem += y[(i, j)] <= x[i]
        problem += y[(i, j)] <= (1e-4 < distances[i, j] <= coverage_distance)

# solver를 통해 최적 해를 도출
problem.solve()

# 도출 결과
print("Status:", pulp.LpStatus[problem.status])
print("Objective:", problem.objective.value())

selected_facilities = [i for i in range(len(facility_points)) if x[i].value() == 1]

# 선정된 최종 후보들의 facility_points 상에서의 인덱스를 출력
print("Selected facilities:", selected_facilities)

covered_demands = [
    (i, j)
    for i in range(len(facility_points))
    for j in range(len(demand_points))
    if y[(i, j)].value() == 1
]
print("Covered demands:", covered_demands)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/bykwon/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/69/8s69_kvj7kq7ycwr88hh27zh0000gn/T/f01ef64b6a97492585351617e275efb5-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/69/8s69_kvj7kq7ycwr88hh27zh0000gn/T/f01ef64b6a97492585351617e275efb5-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 36456 COLUMNS
At line 146212 RHS
At line 182664 BOUNDS
At line 201025 ENDATA
Problem MODEL has 36451 rows, 18360 columns and 54810 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 139 - 0.03 seconds
Cgl0002I 12329 variables fixed
Cgl0004I processed model has 1 rows, 133 columns (133 integer (131 of which binary)) and 133 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0


In [39]:
import folium


geo_json = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'



# Create the map centered at Seoul
m = folium.Map(
    location=[37.566345, 126.977893],
    tiles='Stamen Terrain',
    zoom_start=15
)

# Add GeoJSON data to the map
folium.GeoJson(
    geo_json,
    name='geojson',
    style_function=lambda x: {'fillColor': 'transparent', 'color': 'black', 'weight': 1},
).add_to(m)

# Add demand points to the map
for point in demand_points:
    folium.CircleMarker(
        location=[point[1], point[0]], # point[1] : 경도, point[2] : 위도
        radius=5,
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.7,
    ).add_to(m)

print(covered_demands)

for p in covered_demands:
    print(p[1])
    point = p[1]
    lat = demand_points[point][1]
    lng = demand_points[point][0]
    folium.CircleMarker(
        location=[lat, lng],
        radius=5,
        color="black",
        fill=True,
        fill_color="blue",
        fill_opacity=0.7,
    ).add_to(m)

# 최종 입지의 경도 / 위도 추출
for selected_fac in selected_facilities:
	lng = facility_points[selected_fac][0]
	lat = facility_points[selected_fac][1]

	folium.CircleMarker(
	    location=(lat, lng),
	    radius=5,
	    color="red",
	    fill=True,
	    fill_color="red",
	    fill_opacity=0.7,
	).add_to(m)


	folium.Circle(
	    location=(lat, lng),
	    radius=coverage_distance * 1000,  # Convert kilometers to meters
	    color="green",
	    fill=True,
	    fill_color="green",
	    fill_opacity=0.2,
	).add_to(m)

# Show the map
m

[(37, 0), (37, 1), (37, 2), (37, 3), (37, 4), (37, 5), (37, 9), (37, 13), (37, 14), (37, 15), (37, 17), (37, 18), (37, 19), (37, 24), (37, 25), (37, 26), (37, 27), (37, 28), (37, 31), (37, 33), (37, 34), (37, 35), (37, 36), (37, 38), (37, 39), (37, 40), (37, 41), (37, 42), (37, 43), (37, 44), (37, 45), (37, 46), (37, 47), (37, 48), (37, 49), (37, 58), (37, 59), (37, 60), (37, 67), (37, 68), (37, 71), (37, 72), (37, 73), (37, 74), (37, 77), (37, 78), (37, 79), (37, 86), (37, 87), (37, 91), (37, 92), (37, 93), (37, 98), (37, 103), (37, 110), (37, 111), (37, 112), (37, 113), (37, 114), (37, 116), (37, 117), (37, 118), (37, 123), (37, 124), (37, 125), (37, 126), (37, 127), (37, 128), (37, 129), (37, 130), (37, 132), (73, 2), (73, 3), (73, 5), (73, 6), (73, 9), (73, 12), (73, 13), (73, 15), (73, 16), (73, 17), (73, 18), (73, 19), (73, 22), (73, 25), (73, 26), (73, 27), (73, 28), (73, 31), (73, 33), (73, 36), (73, 37), (73, 38), (73, 39), (73, 40), (73, 41), (73, 42), (73, 43), (73, 44), (73

여의동

In [40]:
demand_points = [(x, y, z) for x, y, z in zip(demands_yu['Longitude'], demands_yu['Latitude'], demands_yu['Demand'])]
facility_points = [(x, y) for x, y in zip(demands_yu['Longitude'], demands_yu['Latitude'])]

In [41]:
import pulp
from haversine import haversine
import numpy as np

# 목적함수를 최대화하는 것을 목표로 함
problem = pulp.LpProblem("MCLP", pulp.LpMaximize)

# 수요 지점과 시설 입지 후보 지점

# 각 시설의 최대 수요 커버 반경 (km 단위)
coverage_distance = 0.5

# 최종적으로 선정할 입지의 갯수
num_facilities = 2

# Haversine을 이용하여 위경도 좌표를 기준으로 거리를 계산
distances = np.array(
    [
        [haversine((fy, fx), (dy, dx)) for dx, dy, _ in demand_points]
        for fx, fy in facility_points
    ]
)

# 각 시설 입지 지점에 대응하는 이진 변수 (x[i] == 1 : 해당 지점이 선택됨 / x[i] == 0 : 해당 지점이 
# 선택되지 않음
x = pulp.LpVariable.dicts(
    "facility", [(i) for i in range(len(facility_points))], cat="Binary"
)

# 각 입지 지점과 수요 지점의 쌍에 대응하는 이진 변수 딕셔너리를 생성
# 이 딕셔너리는 밑에서 추가할 제약을 바탕으로 해당 입지 지점이 선정되었다면, 계산에서 사용됨
y = pulp.LpVariable.dicts(
    "demand",
    [(i, j) for i in range(len(facility_points)) for j in range(len(demand_points))],
    cat="Binary",
)

# 목적 함수 :  sum y_ij * 가중치를 최대화 함
problem += pulp.lpSum(
    [y[(i, j)] * demand_points[j][2] for i in range(len(facility_points)) for j in range(len(demand_points))]
)

# 제약 1 : 최종적으로 선정할 입지의 갯수는 num_facilities와 동일해야 함
problem += pulp.lpSum([x[i] for i in range(len(facility_points))]) == num_facilities

# 제약 2 : y[(i, j)]가 계산되려면, 앞서 x[i] 후보지가 선정되어야 함
# 제약 3 : i번째 후보지와 j번째 수요지 사이의 거리가 coverage_distance보다 작거나 같아야 수요를 커버 가능
for i in range(len(facility_points)):
    for j in range(len(demand_points)):
        problem += y[(i, j)] <= x[i]
        problem += y[(i, j)] <= (1e-4 < distances[i, j] <= coverage_distance)

# solver를 통해 최적 해를 도출
problem.solve()

# 도출 결과
print("Status:", pulp.LpStatus[problem.status])
print("Objective:", problem.objective.value())

selected_facilities = [i for i in range(len(facility_points)) if x[i].value() == 1]

# 선정된 최종 후보들의 facility_points 상에서의 인덱스를 출력
print("Selected facilities:", selected_facilities)

covered_demands = [
    (i, j)
    for i in range(len(facility_points))
    for j in range(len(demand_points))
    if y[(i, j)].value() == 1
]
print("Covered demands:", covered_demands)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/bykwon/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/69/8s69_kvj7kq7ycwr88hh27zh0000gn/T/de2c9b90ecba4fc9b36286a1432d8fe5-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/69/8s69_kvj7kq7ycwr88hh27zh0000gn/T/de2c9b90ecba4fc9b36286a1432d8fe5-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 238056 COLUMNS
At line 953242 RHS
At line 1191294 BOUNDS
At line 1310665 ENDATA
Problem MODEL has 238051 rows, 119370 columns and 357420 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 250 - 0.11 seconds
Cgl0002I 95395 variables fixed
Cgl0004I processed model has 1 rows, 287 columns (287 integer (283 of which binary)) and 287 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied su

In [42]:
import folium


geo_json = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'



# Create the map centered at Seoul
m = folium.Map(
    location=[37.566345, 126.977893],
    tiles='Stamen Terrain',
    zoom_start=15
)

# Add GeoJSON data to the map
folium.GeoJson(
    geo_json,
    name='geojson',
    style_function=lambda x: {'fillColor': 'transparent', 'color': 'black', 'weight': 1},
).add_to(m)

# Add demand points to the map
for point in demand_points:
    folium.CircleMarker(
        location=[point[1], point[0]], # point[1] : 경도, point[2] : 위도
        radius=5,
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.7,
    ).add_to(m)

print(covered_demands)

for p in covered_demands:
    print(p[1])
    point = p[1]
    lat = demand_points[point][1]
    lng = demand_points[point][0]
    folium.CircleMarker(
        location=[lat, lng],
        radius=5,
        color="black",
        fill=True,
        fill_color="blue",
        fill_opacity=0.7,
    ).add_to(m)

# 최종 입지의 경도 / 위도 추출
for selected_fac in selected_facilities:
	lng = facility_points[selected_fac][0]
	lat = facility_points[selected_fac][1]

	folium.CircleMarker(
	    location=(lat, lng),
	    radius=5,
	    color="red",
	    fill=True,
	    fill_color="red",
	    fill_opacity=0.7,
	).add_to(m)


	folium.Circle(
	    location=(lat, lng),
	    radius=coverage_distance * 1000,  # Convert kilometers to meters
	    color="green",
	    fill=True,
	    fill_color="green",
	    fill_opacity=0.2,
	).add_to(m)

# Show the map
m

[(17, 1), (17, 4), (17, 5), (17, 6), (17, 7), (17, 8), (17, 16), (17, 19), (17, 20), (17, 24), (17, 25), (17, 27), (17, 28), (17, 39), (17, 40), (17, 50), (17, 53), (17, 54), (17, 57), (17, 58), (17, 62), (17, 64), (17, 66), (17, 67), (17, 68), (17, 69), (17, 70), (17, 73), (17, 78), (17, 82), (17, 83), (17, 84), (17, 85), (17, 110), (17, 111), (17, 112), (17, 113), (17, 114), (17, 115), (17, 116), (17, 117), (17, 121), (17, 122), (17, 123), (17, 124), (17, 125), (17, 126), (17, 127), (17, 134), (17, 137), (17, 138), (17, 143), (17, 145), (17, 146), (17, 173), (17, 189), (17, 190), (17, 191), (17, 193), (17, 195), (17, 199), (17, 201), (17, 202), (17, 204), (17, 205), (17, 207), (17, 214), (17, 218), (17, 222), (17, 227), (17, 230), (17, 231), (17, 232), (17, 233), (17, 238), (17, 241), (17, 245), (17, 246), (17, 247), (17, 248), (17, 252), (17, 253), (17, 254), (17, 255), (17, 256), (17, 257), (17, 258), (17, 259), (17, 261), (17, 262), (17, 268), (17, 269), (17, 270), (17, 272), (17,

양평2동

In [43]:
demand_points = [(x, y, z) for x, y, z in zip(demands_yp2['Longitude'], demands_yp2['Latitude'], demands_yp2['Demand'])]
facility_points = [(x, y) for x, y in zip(demands_yp2['Longitude'], demands_yp2['Latitude'])]

In [44]:
import pulp
from haversine import haversine
import numpy as np

# 목적함수를 최대화하는 것을 목표로 함
problem = pulp.LpProblem("MCLP", pulp.LpMaximize)

# 수요 지점과 시설 입지 후보 지점

# 각 시설의 최대 수요 커버 반경 (km 단위)
coverage_distance = 0.5

# 최종적으로 선정할 입지의 갯수
num_facilities = 2

# Haversine을 이용하여 위경도 좌표를 기준으로 거리를 계산
distances = np.array(
    [
        [haversine((fy, fx), (dy, dx)) for dx, dy, _ in demand_points]
        for fx, fy in facility_points
    ]
)

# 각 시설 입지 지점에 대응하는 이진 변수 (x[i] == 1 : 해당 지점이 선택됨 / x[i] == 0 : 해당 지점이 
# 선택되지 않음
x = pulp.LpVariable.dicts(
    "facility", [(i) for i in range(len(facility_points))], cat="Binary"
)

# 각 입지 지점과 수요 지점의 쌍에 대응하는 이진 변수 딕셔너리를 생성
# 이 딕셔너리는 밑에서 추가할 제약을 바탕으로 해당 입지 지점이 선정되었다면, 계산에서 사용됨
y = pulp.LpVariable.dicts(
    "demand",
    [(i, j) for i in range(len(facility_points)) for j in range(len(demand_points))],
    cat="Binary",
)

# 목적 함수 :  sum y_ij * 가중치를 최대화 함
problem += pulp.lpSum(
    [y[(i, j)] * demand_points[j][2] for i in range(len(facility_points)) for j in range(len(demand_points))]
)

# 제약 1 : 최종적으로 선정할 입지의 갯수는 num_facilities와 동일해야 함
problem += pulp.lpSum([x[i] for i in range(len(facility_points))]) == num_facilities

# 제약 2 : y[(i, j)]가 계산되려면, 앞서 x[i] 후보지가 선정되어야 함
# 제약 3 : i번째 후보지와 j번째 수요지 사이의 거리가 coverage_distance보다 작거나 같아야 수요를 커버 가능
for i in range(len(facility_points)):
    for j in range(len(demand_points)):
        problem += y[(i, j)] <= x[i]
        problem += y[(i, j)] <= (1e-4 < distances[i, j] <= coverage_distance)

# solver를 통해 최적 해를 도출
problem.solve()

# 도출 결과
print("Status:", pulp.LpStatus[problem.status])
print("Objective:", problem.objective.value())

selected_facilities = [i for i in range(len(facility_points)) if x[i].value() == 1]

# 선정된 최종 후보들의 facility_points 상에서의 인덱스를 출력
print("Selected facilities:", selected_facilities)

covered_demands = [
    (i, j)
    for i in range(len(facility_points))
    for j in range(len(demand_points))
    if y[(i, j)].value() == 1
]
print("Covered demands:", covered_demands)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/bykwon/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/69/8s69_kvj7kq7ycwr88hh27zh0000gn/T/4ce72c25a2654f7aad0b82a0a0f6eb71-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/69/8s69_kvj7kq7ycwr88hh27zh0000gn/T/4ce72c25a2654f7aad0b82a0a0f6eb71-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 25094 COLUMNS
At line 100695 RHS
At line 125785 BOUNDS
At line 138442 ENDATA
Problem MODEL has 25089 rows, 12656 columns and 37744 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 130 - 0.02 seconds
Cgl0002I 7684 variables fixed
Cgl0004I processed model has 1 rows, 105 columns (105 integer (103 of which binary)) and 105 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
C

In [45]:
import folium


geo_json = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'



# Create the map centered at Seoul
m = folium.Map(
    location=[37.566345, 126.977893],
    tiles='Stamen Terrain',
    zoom_start=15
)

# Add GeoJSON data to the map
folium.GeoJson(
    geo_json,
    name='geojson',
    style_function=lambda x: {'fillColor': 'transparent', 'color': 'black', 'weight': 1},
).add_to(m)

# Add demand points to the map
for point in demand_points:
    folium.CircleMarker(
        location=[point[1], point[0]], # point[1] : 경도, point[2] : 위도
        radius=5,
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.7,
    ).add_to(m)

print(covered_demands)

for p in covered_demands:
    print(p[1])
    point = p[1]
    lat = demand_points[point][1]
    lng = demand_points[point][0]
    folium.CircleMarker(
        location=[lat, lng],
        radius=5,
        color="black",
        fill=True,
        fill_color="blue",
        fill_opacity=0.7,
    ).add_to(m)

# 최종 입지의 경도 / 위도 추출
for selected_fac in selected_facilities:
	lng = facility_points[selected_fac][0]
	lat = facility_points[selected_fac][1]

	folium.CircleMarker(
	    location=(lat, lng),
	    radius=5,
	    color="red",
	    fill=True,
	    fill_color="red",
	    fill_opacity=0.7,
	).add_to(m)


	folium.Circle(
	    location=(lat, lng),
	    radius=coverage_distance * 1000,  # Convert kilometers to meters
	    color="green",
	    fill=True,
	    fill_color="green",
	    fill_opacity=0.2,
	).add_to(m)

# Show the map
m

[(29, 0), (29, 5), (29, 6), (29, 14), (29, 16), (29, 17), (29, 18), (29, 20), (29, 21), (29, 22), (29, 26), (29, 27), (29, 28), (29, 30), (29, 31), (29, 35), (29, 36), (29, 37), (29, 42), (29, 44), (29, 45), (29, 46), (29, 48), (29, 49), (29, 50), (29, 51), (29, 52), (29, 54), (29, 55), (29, 56), (29, 57), (29, 64), (29, 65), (29, 66), (29, 67), (29, 69), (29, 71), (29, 72), (29, 73), (29, 74), (29, 75), (29, 76), (29, 79), (29, 80), (29, 83), (29, 84), (29, 85), (29, 86), (29, 87), (29, 88), (29, 90), (29, 91), (29, 92), (29, 93), (29, 94), (29, 101), (29, 102), (29, 103), (29, 104), (29, 105), (29, 106), (29, 107), (29, 108), (29, 109), (74, 1), (74, 2), (74, 3), (74, 4), (74, 5), (74, 6), (74, 16), (74, 17), (74, 20), (74, 21), (74, 22), (74, 24), (74, 25), (74, 26), (74, 27), (74, 28), (74, 29), (74, 30), (74, 35), (74, 36), (74, 37), (74, 38), (74, 39), (74, 43), (74, 44), (74, 45), (74, 46), (74, 47), (74, 48), (74, 49), (74, 50), (74, 51), (74, 52), (74, 53), (74, 54), (74, 55),

문래동

In [46]:
demand_points = [(x, y, z) for x, y, z in zip(demands_ml['Longitude'], demands_ml['Latitude'], demands_ml['Demand'])]
facility_points = [(x, y) for x, y in zip(demands_ml['Longitude'], demands_ml['Latitude'])]

In [47]:
import pulp
from haversine import haversine
import numpy as np

# 목적함수를 최대화하는 것을 목표로 함
problem = pulp.LpProblem("MCLP", pulp.LpMaximize)

# 수요 지점과 시설 입지 후보 지점

# 각 시설의 최대 수요 커버 반경 (km 단위)
coverage_distance = 0.5

# 최종적으로 선정할 입지의 갯수
num_facilities = 2

# Haversine을 이용하여 위경도 좌표를 기준으로 거리를 계산
distances = np.array(
    [
        [haversine((fy, fx), (dy, dx)) for dx, dy, _ in demand_points]
        for fx, fy in facility_points
    ]
)

# 각 시설 입지 지점에 대응하는 이진 변수 (x[i] == 1 : 해당 지점이 선택됨 / x[i] == 0 : 해당 지점이 
# 선택되지 않음
x = pulp.LpVariable.dicts(
    "facility", [(i) for i in range(len(facility_points))], cat="Binary"
)

# 각 입지 지점과 수요 지점의 쌍에 대응하는 이진 변수 딕셔너리를 생성
# 이 딕셔너리는 밑에서 추가할 제약을 바탕으로 해당 입지 지점이 선정되었다면, 계산에서 사용됨
y = pulp.LpVariable.dicts(
    "demand",
    [(i, j) for i in range(len(facility_points)) for j in range(len(demand_points))],
    cat="Binary",
)

# 목적 함수 :  sum y_ij * 가중치를 최대화 함
problem += pulp.lpSum(
    [y[(i, j)] * demand_points[j][2] for i in range(len(facility_points)) for j in range(len(demand_points))]
)

# 제약 1 : 최종적으로 선정할 입지의 갯수는 num_facilities와 동일해야 함
problem += pulp.lpSum([x[i] for i in range(len(facility_points))]) == num_facilities

# 제약 2 : y[(i, j)]가 계산되려면, 앞서 x[i] 후보지가 선정되어야 함
# 제약 3 : i번째 후보지와 j번째 수요지 사이의 거리가 coverage_distance보다 작거나 같아야 수요를 커버 가능
for i in range(len(facility_points)):
    for j in range(len(demand_points)):
        problem += y[(i, j)] <= x[i]
        problem += y[(i, j)] <= (1e-4 < distances[i, j] <= coverage_distance)

# solver를 통해 최적 해를 도출
problem.solve()

# 도출 결과
print("Status:", pulp.LpStatus[problem.status])
print("Objective:", problem.objective.value())

selected_facilities = [i for i in range(len(facility_points)) if x[i].value() == 1]

# 선정된 최종 후보들의 facility_points 상에서의 인덱스를 출력
print("Selected facilities:", selected_facilities)

covered_demands = [
    (i, j)
    for i in range(len(facility_points))
    for j in range(len(demand_points))
    if y[(i, j)].value() == 1
]
print("Covered demands:", covered_demands)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/bykwon/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/69/8s69_kvj7kq7ycwr88hh27zh0000gn/T/9c5edb0a034d4283aab15eecb024f2f9-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/69/8s69_kvj7kq7ycwr88hh27zh0000gn/T/9c5edb0a034d4283aab15eecb024f2f9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 51848 COLUMNS
At line 207858 RHS
At line 259702 BOUNDS
At line 285785 ENDATA
Problem MODEL has 51843 rows, 26082 columns and 77924 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 167 - 0.05 seconds
Cgl0002I 16961 variables fixed
Cgl0004I processed model has 1 rows, 141 columns (141 integer (137 of which binary)) and 141 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0


In [48]:
import folium


geo_json = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'



# Create the map centered at Seoul
m = folium.Map(
    location=[37.566345, 126.977893],
    tiles='Stamen Terrain',
    zoom_start=15
)

# Add GeoJSON data to the map
folium.GeoJson(
    geo_json,
    name='geojson',
    style_function=lambda x: {'fillColor': 'transparent', 'color': 'black', 'weight': 1},
).add_to(m)

# Add demand points to the map
for point in demand_points:
    folium.CircleMarker(
        location=[point[1], point[0]], # point[1] : 경도, point[2] : 위도
        radius=5,
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.7,
    ).add_to(m)

print(covered_demands)

for p in covered_demands:
    print(p[1])
    point = p[1]
    lat = demand_points[point][1]
    lng = demand_points[point][0]
    folium.CircleMarker(
        location=[lat, lng],
        radius=5,
        color="black",
        fill=True,
        fill_color="blue",
        fill_opacity=0.7,
    ).add_to(m)

# 최종 입지의 경도 / 위도 추출
for selected_fac in selected_facilities:
	lng = facility_points[selected_fac][0]
	lat = facility_points[selected_fac][1]

	folium.CircleMarker(
	    location=(lat, lng),
	    radius=5,
	    color="red",
	    fill=True,
	    fill_color="red",
	    fill_opacity=0.7,
	).add_to(m)


	folium.Circle(
	    location=(lat, lng),
	    radius=coverage_distance * 1000,  # Convert kilometers to meters
	    color="green",
	    fill=True,
	    fill_color="green",
	    fill_opacity=0.2,
	).add_to(m)

# Show the map
m

[(37, 1), (37, 2), (37, 6), (37, 7), (37, 8), (37, 9), (37, 10), (37, 13), (37, 14), (37, 15), (37, 17), (37, 18), (37, 19), (37, 20), (37, 22), (37, 24), (37, 29), (37, 35), (37, 36), (37, 38), (37, 39), (37, 40), (37, 43), (37, 44), (37, 47), (37, 48), (37, 49), (37, 50), (37, 52), (37, 53), (37, 54), (37, 55), (37, 56), (37, 57), (37, 58), (37, 60), (37, 61), (37, 62), (37, 65), (37, 68), (37, 69), (37, 74), (37, 75), (37, 76), (37, 80), (37, 81), (37, 82), (37, 84), (37, 88), (37, 89), (37, 91), (37, 92), (37, 94), (37, 96), (37, 98), (37, 99), (37, 100), (37, 102), (37, 105), (37, 106), (37, 109), (37, 110), (37, 111), (37, 115), (37, 117), (37, 118), (37, 119), (37, 120), (37, 128), (37, 129), (37, 130), (37, 131), (37, 132), (37, 133), (37, 139), (37, 140), (37, 143), (37, 144), (37, 145), (37, 146), (37, 147), (37, 151), (111, 1), (111, 2), (111, 6), (111, 7), (111, 8), (111, 9), (111, 10), (111, 13), (111, 14), (111, 15), (111, 17), (111, 18), (111, 19), (111, 20), (111, 22), 

대림3동

In [49]:

demand_points = [(x, y, z) for x, y, z in zip(demands_dl3['Longitude'], demands_dl3['Latitude'], demands_dl3['Demand'])]
facility_points = [(x, y) for x, y in zip(demands_dl3['Longitude'], demands_dl3['Latitude'])]

In [50]:
import pulp
from haversine import haversine
import numpy as np

# 목적함수를 최대화하는 것을 목표로 함
problem = pulp.LpProblem("MCLP", pulp.LpMaximize)

# 수요 지점과 시설 입지 후보 지점

# 각 시설의 최대 수요 커버 반경 (km 단위)
coverage_distance = 0.5

# 최종적으로 선정할 입지의 갯수
num_facilities = 2

# Haversine을 이용하여 위경도 좌표를 기준으로 거리를 계산
distances = np.array(
    [
        [haversine((fy, fx), (dy, dx)) for dx, dy, _ in demand_points]
        for fx, fy in facility_points
    ]
)

# 각 시설 입지 지점에 대응하는 이진 변수 (x[i] == 1 : 해당 지점이 선택됨 / x[i] == 0 : 해당 지점이 
# 선택되지 않음
x = pulp.LpVariable.dicts(
    "facility", [(i) for i in range(len(facility_points))], cat="Binary"
)

# 각 입지 지점과 수요 지점의 쌍에 대응하는 이진 변수 딕셔너리를 생성
# 이 딕셔너리는 밑에서 추가할 제약을 바탕으로 해당 입지 지점이 선정되었다면, 계산에서 사용됨
y = pulp.LpVariable.dicts(
    "demand",
    [(i, j) for i in range(len(facility_points)) for j in range(len(demand_points))],
    cat="Binary",
)

# 목적 함수 :  sum y_ij * 가중치를 최대화 함
problem += pulp.lpSum(
    [y[(i, j)] * demand_points[j][2] for i in range(len(facility_points)) for j in range(len(demand_points))]
)

# 제약 1 : 최종적으로 선정할 입지의 갯수는 num_facilities와 동일해야 함
problem += pulp.lpSum([x[i] for i in range(len(facility_points))]) == num_facilities

# 제약 2 : y[(i, j)]가 계산되려면, 앞서 x[i] 후보지가 선정되어야 함
# 제약 3 : i번째 후보지와 j번째 수요지 사이의 거리가 coverage_distance보다 작거나 같아야 수요를 커버 가능
for i in range(len(facility_points)):
    for j in range(len(demand_points)):
        problem += y[(i, j)] <= x[i]
        problem += y[(i, j)] <= (1e-4 < distances[i, j] <= coverage_distance)

# solver를 통해 최적 해를 도출
problem.solve()

# 도출 결과
print("Status:", pulp.LpStatus[problem.status])
print("Objective:", problem.objective.value())

selected_facilities = [i for i in range(len(facility_points)) if x[i].value() == 1]

# 선정된 최종 후보들의 facility_points 상에서의 인덱스를 출력
print("Selected facilities:", selected_facilities)

covered_demands = [
    (i, j)
    for i in range(len(facility_points))
    for j in range(len(demand_points))
    if y[(i, j)].value() == 1
]
print("Covered demands:", covered_demands)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/bykwon/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/69/8s69_kvj7kq7ycwr88hh27zh0000gn/T/a01abb115bae415d853c8d1d0208a731-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/69/8s69_kvj7kq7ycwr88hh27zh0000gn/T/a01abb115bae415d853c8d1d0208a731-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 18438 COLUMNS
At line 74023 RHS
At line 92457 BOUNDS
At line 101770 ENDATA
Problem MODEL has 18433 rows, 9312 columns and 27744 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 127 - 0.01 seconds
Cgl0002I 4968 variables fixed
Cgl0004I processed model has 1 rows, 49 columns (49 integer (45 of which binary)) and 49 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I

In [51]:
import folium


geo_json = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'



# Create the map centered at Seoul
m = folium.Map(
    location=[37.566345, 126.977893],
    tiles='Stamen Terrain',
    zoom_start=15
)

# Add GeoJSON data to the map
folium.GeoJson(
    geo_json,
    name='geojson',
    style_function=lambda x: {'fillColor': 'transparent', 'color': 'black', 'weight': 1},
).add_to(m)

# Add demand points to the map
for point in demand_points:
    folium.CircleMarker(
        location=[point[1], point[0]], # point[1] : 경도, point[2] : 위도
        radius=5,
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.7,
    ).add_to(m)

print(covered_demands)

for p in covered_demands:
    print(p[1])
    point = p[1]
    lat = demand_points[point][1]
    lng = demand_points[point][0]
    folium.CircleMarker(
        location=[lat, lng],
        radius=5,
        color="black",
        fill=True,
        fill_color="blue",
        fill_opacity=0.7,
    ).add_to(m)

# 최종 입지의 경도 / 위도 추출
for selected_fac in selected_facilities:
	lng = facility_points[selected_fac][0]
	lat = facility_points[selected_fac][1]

	folium.CircleMarker(
	    location=(lat, lng),
	    radius=5,
	    color="red",
	    fill=True,
	    fill_color="red",
	    fill_opacity=0.7,
	).add_to(m)


	folium.Circle(
	    location=(lat, lng),
	    radius=coverage_distance * 1000,  # Convert kilometers to meters
	    color="green",
	    fill=True,
	    fill_color="green",
	    fill_opacity=0.2,
	).add_to(m)

# Show the map
m

[(35, 1), (35, 2), (35, 3), (35, 4), (35, 5), (35, 6), (35, 10), (35, 11), (35, 12), (35, 13), (35, 15), (35, 17), (35, 18), (35, 19), (35, 20), (35, 25), (35, 26), (35, 27), (35, 28), (35, 29), (35, 31), (35, 32), (35, 33), (35, 34), (35, 36), (35, 37), (35, 38), (35, 40), (35, 42), (35, 43), (35, 44), (35, 45), (35, 47), (35, 48), (35, 51), (35, 52), (35, 53), (35, 54), (35, 57), (35, 58), (35, 59), (35, 60), (35, 65), (35, 66), (35, 68), (35, 69), (35, 70), (35, 71), (35, 72), (35, 73), (35, 74), (35, 76), (35, 77), (35, 79), (35, 80), (35, 81), (35, 82), (35, 85), (35, 87), (35, 88), (35, 91), (35, 93), (35, 95), (91, 1), (91, 2), (91, 3), (91, 4), (91, 5), (91, 6), (91, 10), (91, 11), (91, 12), (91, 13), (91, 15), (91, 17), (91, 18), (91, 19), (91, 20), (91, 25), (91, 26), (91, 27), (91, 28), (91, 29), (91, 31), (91, 32), (91, 33), (91, 34), (91, 35), (91, 36), (91, 37), (91, 38), (91, 40), (91, 42), (91, 43), (91, 44), (91, 45), (91, 47), (91, 48), (91, 51), (91, 52), (91, 53), (